In [1]:
from util import parse_db, Document, read_markup_tsv
import random
from torch.utils.data import Dataset, DataLoader
import sentence_transformers

records = parse_db("data_/0525_parsed.db")


In [79]:
import torch
from sentence_transformers import CrossEncoder
from torch.utils.data import DataLoader

model = CrossEncoder('bert-base-multilingual-uncased', num_labels= 1)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [12]:
from sentence_transformers import InputExample

random.seed(0)

def separate_samples(markup, train_percent=0.85):

    count_samples = len(markup)
    shuffled_ids = list(range(count_samples))
    random.shuffle(shuffled_ids)
    train_len = round(count_samples * train_percent)

    train_markup, test_markup = [], []

    for i, id in enumerate(shuffled_ids):
      if i < train_len:
        train_markup.append(markup[id])
      else:
        test_markup.append(markup[id])        
         
    return train_markup, test_markup


def get_data(markup, records):
    input_samples, other_samples, qualities = [], [], []

    for mrkp in markup:
        first_url, second_url, quality = mrkp.values()

        input_samples.append(url2record[first_url]['title'] + ' ' + url2record[first_url]['text'])
        other_samples.append(url2record[second_url]['title'] + ' ' + url2record[second_url]['text'])
        if quality == 'OK':
            qualities.append(1)
        else:
            qualities.append(0)
    return input_samples, other_samples, qualities

url2record = dict()

for i, record in enumerate(records):
    url2record[record["url"]] = record


markup = read_markup_tsv("data_/ru_clustering_0525_urls.tsv")
train_markup, test_markup = separate_samples(markup)
train_inputs, train_others, train_qualities  = get_data(train_markup, records)
test_inputs, test_others, test_qualities  = get_data(test_markup, records)

In [13]:
train_examples = [InputExample(texts=[inp, oth], label = qual) for inp, oth, qual in zip(train_inputs, train_others, train_qualities)]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

test_examples = [InputExample(texts=[inp, oth], label = qual) for inp, oth, qual in zip(test_inputs, test_others, test_qualities)]
test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=16)


In [14]:
from sentence_transformers.cross_encoder.evaluation import CEBinaryAccuracyEvaluator
evaluator = CEBinaryAccuracyEvaluator(list(zip(test_inputs, test_others)), test_qualities)

In [19]:
res = model.predict(list(zip(test_inputs, test_others)))

In [20]:
import sklearn
import numpy as np
sklearn.metrics.f1_score(test_qualities, np.expand_dims(res, axis=1).round(), pos_label=1, average='binary')

0.9629959875167188

In [83]:
model.fit(train_dataloader=train_dataloader,
         evaluator = evaluator,
         warmup_steps = 600,
         epochs=10)

I0302 01:13:57.936445  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 0:


I0302 01:14:11.493777  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 95.96


I0302 01:18:33.853937  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 1:


I0302 01:18:47.414166  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.18


I0302 01:23:09.296399  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 2:


...

I0302 01:46:43.397601  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.68


I0302 01:51:13.278924  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 8:


I0302 01:51:27.464131  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.54


I0302 01:55:57.357455  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 9:


I0302 01:56:11.531880  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.63


In [87]:
model = CrossEncoder('bert-base-multilingual-cased', num_labels= 1)

I0302 02:09:58.988414  3016 filelock.py:274] Lock 3068270013464 acquired on C:\Users\Ellie/.cache\huggingface\transformers\6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c.lock


I0302 02:09:59.546783  3016 filelock.py:318] Lock 3068270013464 released on C:\Users\Ellie/.cache\huggingface\transformers\6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c.lock


I0302 02:10:00.115547  3016 filelock.py:274] Lock 3068270013744 acquired on C:\Users\Ellie/.cache\huggingface\transformers\0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052.lock


I0302 02:10:55.171320  3016 filelock.py:318] Lock 3068270013744 released on C:\Users\Ellie/.cache\huggingface\transformers\0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052.lock


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

I0302 02:11:00.970126  3016 filelock.py:318] Lock 3068748115136 released on C:\Users\Ellie/.cache\huggingface\transformers\eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29.lock


I0302 02:11:01.529527  3016 filelock.py:274] Lock 3068748115136 acquired on C:\Users\Ellie/.cache\huggingface\transformers\46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24.lock


I0302 02:11:03.050669  3016 filelock.py:318] Lock 3068748115136 released on C:\Users\Ellie/.cache\huggingface\transformers\46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24.lock
I0302 02:11:03.116493  3016 CrossEncoder.py:54] Use pytorch device: cuda


In [91]:
model.fit(train_dataloader=train_dataloader,
         evaluator = evaluator,
         warmup_steps = 600,
         epochs=10)

I0302 02:17:00.094772  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 0:


I0302 02:17:13.662716  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 95.78


I0302 02:21:38.302990  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 1:


I0302 02:21:52.157829  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 95.82


I0302 02:26:15.667087  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 2:


...

I0302 02:49:26.678188  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.09


I0302 02:53:49.341983  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 8:


I0302 02:54:02.897368  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.14


I0302 02:58:25.889133  3016 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 9:


I0302 02:58:39.484540  3016 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.27


In [8]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('xlm-roberta-base', num_labels= 1)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [9]:
model.fit(train_dataloader=train_dataloader,
         evaluator = evaluator,
         warmup_steps = 600,
         epochs=10)

I0302 03:18:43.352751  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 0:


I0302 03:18:56.892196  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 94.29


I0302 03:23:22.554132  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 1:


I0302 03:23:36.073270  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 95.33


I0302 03:28:01.258845  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 2:


...

I0302 03:51:23.800127  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.32


I0302 03:55:47.733868  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 8:


I0302 03:56:01.270124  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.14


I0302 04:00:25.120123  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 9:


I0302 04:00:38.674279  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.14


In [17]:
import torch
torch.cuda.empty_cache()
model = CrossEncoder('xlm-roberta-base', num_labels= 1)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [18]:
model.fit(train_dataloader=train_dataloader,
         evaluator = evaluator,
         warmup_steps = 600,
         epochs=10)

I0302 09:43:09.697994  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 0:


I0302 09:43:23.280348  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 94.97


I0302 09:47:50.926039  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 1:


I0302 09:48:04.566362  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 95.19


I0302 09:52:31.258478  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 2:


...

I0302 10:16:06.655925  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.32


I0302 10:20:32.004658  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 8:


I0302 10:20:45.512651  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.18


I0302 10:25:10.969971  7160 CEBinaryAccuracyEvaluator.py:48] CESoftmaxAccuracyEvaluator: Evaluating the model on  dataset after epoch 9:


I0302 10:25:24.485972  7160 CEBinaryAccuracyEvaluator.py:56] Accuracy: 96.27
